
# Step 1: Build a dummy Model

Let's start by making an assumption that a house price in rural area costs 1500 per meter square and a house in the city costs 5000 per meter square from this assumption we will generate some dummy data.

In [16]:
!pip install tensorflow==2.11.0
!pip install -U ipykernel

     |████████████████████████████████| 152 kB 6.6 MB/s eta 0:00:01
ERROR: responsibleai 0.24.0 has requirement ipykernel<=6.6.0, but you'll have ipykernel 6.23.3 which is incompatible.
ERROR: responsibleai 0.24.0 has requirement markupsafe<2.1.0, but you'll have markupsafe 2.1.3 which is incompatible.
ERROR: responsibleai 0.24.0 has requirement numba<0.54.0, but you'll have numba 0.55.2 which is incompatible.
ERROR: raiwidgets 0.24.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.2.1 which is incompatible.
ERROR: jupyter-console 6.6.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 6.1.12 which is incompatible.
ERROR: azureml-widgets 1.49.0 has requirement markupsafe<2.1.0, but you'll have markupsafe 2.1.3 which is incompatible.
ERROR: azureml-contrib-notebook 1.49.0 has requirement nbconvert<6, but you'll have nbconvert 7.

In [1]:
# Import needed libraries
import tensorflow as tf
import numpy as np
from tensorflow import keras

## 1.a Create Normalized Dummy Data

In [2]:
# First Column number of bedrooms
# Second Column Area in Square meter divided by 100
# Third Column Rural or City location 1 represents city and 0 for Rural
xs = np.array([['1.0', '1.0', '0.0'],
               ['1.0', '0.5', '1.0'],
               ['2.0', '2.0', '0.0'],
               ['2.0', '1.0', '1.0'],
               ['3.0', '3.0', '0.0'],
               ['4.0', '4.0', '1.0'],
               ['6.0', '6.0', '0.0'],
               ['3.0', '3.0', '1.0']
             ], dtype=float)
# Output label that we are trying to predict divded by 100,000
ys = np.array(['1.5', '2.5', '3.0', '5.0', '4.5', '20.0', '9.0', '15.0'], dtype=float)

## 1.b Create the Machine Learning Model

In [3]:
# A simple regression model created using tensorflow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=[3]),
    tf.keras.layers.Dense(1, activation='relu'),
    ])
# We are using mean squared error loss and stochastic gradient descent optimizer
model.compile(optimizer='sgd',
              loss='mse',
              metrics=['mae', 'mse'])

2023-06-29 03:24:46.242078: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-06-29 03:24:46.576128: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-29 03:24:46.576183: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (testing-computer-123abc): /proc/driver/nvidia/version does not exist
2023-06-29 03:24:46.578504: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-06-29 03:24:46.675895: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2593905000 Hz
2023-06-29 03:24:46.676247: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7254000b60 initialized for platform Host (this does not guarantee that XLA will be used). D

In [4]:
# Start the training to fit the input data to the target data 1000 times
model.fit(xs, ys, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 83.8261 - mae: 6.8880 - mse: 83.8261
Epoch 2/1000
1/1 [==============================] - 0s 970us/step - loss: 35.5890 - mae: 3.6710 - mse: 35.5890
Epoch 3/1000
1/1 [==============================] - 0s 797us/step - loss: 25.7146 - mae: 3.9910 - mse: 25.7146
Epoch 4/1000
1/1 [==============================] - 0s 838us/step - loss: 25.1201 - mae: 3.8697 - mse: 25.1201
Epoch 5/1000
1/1 [==============================] - 0s 829us/step - loss: 24.6134 - mae: 3.8579 - mse: 24.6134
Epoch 6/1000
1/1 [==

## 1.c Test the Model

In [7]:
# Make a prediction
prediction = model.predict([[2.0, 1.0, 1.0]])

In [8]:
# Convert the prediction from normalized to real value
print(round(round(prediction[0][0], 1)* 100000, 1))

470000.0


**Note: This model is not accurate it may hit 1 time and miss 10 times it's just for illustration.**

**You should replace it with your own model**

## 1.d Save the Model

In [10]:
model.save('./tf-model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./tf-model/assets


2023-06-29 03:26:56.807845: W tensorflow/python/util/util.cc:329] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


# Step 2: Package the Model using ONNX

ONNX is an open format built to represent machine learning models. ONNX defines a common set of operators - the building blocks of machine learning and deep learning models - and a common file format to enable AI developers to use models with a variety of frameworks, tools, runtimes, and compilers.

You may build your model using tensorflow, pytorch, etc.. but in the end you can use onnx as a unified format for your deployment which makes the deployment process much more easier.

## 2.a Install needed libraries

In [11]:
!pip install onnxruntime

In [12]:
!pip install -U tf2onnx

     |████████████████████████████████| 451 kB 6.3 MB/s eta 0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.1.21
    Uninstalling flatbuffers-23.1.21:
      Successfully uninstalled flatbuffers-23.1.21


## 2.b Convert the model to ONNX format

If your model is not a tensorflow model no worries.

See the guide here ==> https://github.com/onnx/tutorials#converting-to-onnx-format

In [13]:
!python -m tf2onnx.convert --saved-model ./tf-model --output ./model/house_price_model.onnx

/anaconda/envs/azureml_py38/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-06-29 03:27:35,170 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2023-06-29 03:27:41,496 - INFO - Signatures found in model: [serving_default].
2023-06-29 03:27:41,496 - WARNING - '--signature_def' not specified, using first signature: serving_default
2023-06-29 03:27:41,496 - INFO - Output names: ['dense_1']
2023-06-29 03:27:41,556 - INFO - Using tensorflow=2.2.1, onnx=1.12.0, tf2onnx=1.14.0/8f8d49
2023-06-29 03:27:41,556 - INFO - Using opset <onnx, 15>
2023-06-29 03:27:41,560 - INFO - Computed 0 values for constant folding
2023-06-29 03:27:41,568 - INFO - Optimizing ONNX model
2023-06-29 03:27:41,591 - INFO - After optimization: Identity -2 (2->0)
2023-06-29 03:27:42,466 - INFO - 
2023-06-29 03:27:4

## 2.c Test the ONNX format

Let's confirm that ONNX model produces the same output

In [14]:
import numpy as np
import onnxruntime as rt

# Load the onnx model
onnx_session = rt.InferenceSession('./model/house_price_model.onnx')

In [15]:
# Create a sample from the data
input_data =  [[[2.0, 1.0, 1.0]]]

# Prepare the data in the shape that onnx accepts
feed = dict([(input.name, input_data[n]) for n, input in enumerate(onnx_session.get_inputs())]) # {'dense_10_input': [[2.0, 1.0, 1.0]]}

In [16]:
# Make a prediction
prediction = onnx_session.run(None, feed)[0][0][0]

# Convert the prediction from normalized to real value
print(round(round(prediction, 1)* 100000, 1))

470000.0


# Step 3: Deploy the Model

Before we deploy the model we need 2 files:


*   Score.py (needed to load the model and make a prediction when anyone invokes our model endpoint)
*   myenv.yml (our environment dependencies which are the libraries needed to run our model)



## 3.a Create the Environment Dependencies file

In [17]:
%%writefile ./model/myenv.yml
name: inference_environment
dependencies:
- python=3.8.5
- pip
- pip:
    - azureml-defaults==1.49.0
    - pillow==9.2.0
    - onnxruntime==1.11.1
    - azureml-contrib-services==1.49.0
    - numpy==1.21.6

Writing ./model/myenv.yml


## 3.b Create the Scoring file

In [30]:
%%writefile ./model/score.py
import json
import logging
import os
import numpy as np
import onnxruntime as rt

from azureml.core.model import Model
from azureml.contrib.services.aml_response import AMLResponse



def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global session
    model_dir = Model.get_model_path(model_name="HousePricePrediction-secure")
    # the name of the folder
    model_filename = 'house_price_model.onnx'
    model_path = os.path.join(model_dir, model_filename)

    session = rt.InferenceSession(model_path)
    logging.info("Init complete")


def run(raw_data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    """
    logging.info("model 1: request received")
    data = json.loads(raw_data)['input_data']
    input_data = np.array(data, dtype=np.float32)
    if input_data is not None:
        feed = dict([(input.name, input_data[n]) for n, input in enumerate(session.get_inputs())])
        prediction = session.run(None, feed)[0]
        return AMLResponse(json.dumps({'output_data':prediction.tolist()}), 200)
    else:
        return AMLResponse("bad request", 400)


Overwriting ./model/score.py


## 3.c Import needed Libraries

In [1]:
import json
import os
import requests

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Environment

from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

## 3.d Initialize Needed Variables

In [2]:
# The name of the model as it will appear in AzureML
aml_model_name = 'HousePricePrediction-secure'  # Updating this requires an update to score.py

# The name of the model endpoint to be created in AzureML
aci_service_name = 'house-price-prediction-onnx'

# The name of the model as it will appear in AI Builder
aib_model_name = "house-price-prediction-v1"

# The local path of the parent of the model directory
model_path = '.'

is_secure = True

In [ ]:
# Get your workspace Configuration varibales
ws = Workspace.from_config()

## 3.e Register the ML Model in Azure ML

In [4]:
# Register an AML Model
model_root = os.path.join(model_path, './model')
model = Model.register(workspace=ws,
                       model_path=model_root,
                       model_name=aml_model_name,
                       tags={'area': "numbers", 'type': "regression"},
                       )

print(f"Registered model {model.name}, Version {model.version}")

Registering model HousePricePrediction-secure
Registered model HousePricePrediction-secure, Version 7


## 3.f Create the Deployment Endpoint

In [5]:
#Creating an inference configuration

entry_script = os.path.join(model_root, "score.py")
conda_file = os.path.join(model_root, "myenv.yml")

inference_config = InferenceConfig(runtime="python",
                                   entry_script=entry_script,
                                   conda_file=conda_file)

In [ ]:
service = None

try:
    # get any existing service with the specified name
    service = Webservice(ws, name=aci_service_name)
except WebserviceException as e:
    print(f"Webservice not found: {aci_service_name}")

# Update the service with the new model if the service exists, otherwise deploy a new service
if service:
    print (f"Updating service {aci_service_name}")
    model = Model(workspace=ws, name=aml_model_name)
    service.update(models=[model], inference_config=inference_config, auth_enabled=is_secure)
else:
    print (f"Deploying new service {aci_service_name}")
    deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.8, memory_gb = 1, auth_enabled=is_secure)
    service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

## 3.g Test the Deployed Model

In [7]:
#validate service using response
service = Webservice(ws, name=aci_service_name)
uri = service.scoring_uri
input_data =  [[[2.0, 1.0, 1.0]]]
api_key = service.get_keys()[0] # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

request_data = json.dumps({'input_data': input_data})

In [8]:
# Send the Request
response = requests.post(uri, headers=headers, data=request_data)

In [ ]:
print("ML API Key: {}\nML Endpoint: {}".format(service.get_keys()[0], service.scoring_uri))